# Lecture - MediaWiki API

Author: Jun Sun (jun.sun@gesis.org)

In this notebook, we will learn how to use the MediaWiki API to 

1. search the Wikipedia
2. fetch page content from Wikipedia

In [1]:
import pandas as pd # for data manipulation
import requests     # for executing the HTTP request
import json         # for data manipulation in JSON format
import time         # api's not designed for big data analysis so we need to regulate our requests
import datetime     # converting UNIX timestamps to human readable date formats

from IPython.display import Javascript, HTML

## Search the English Wikipedia with keyword "mannheim"

In [2]:
url = "https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch=mannheim&format=json"
r = requests.get(url)
data = json.loads(r.text)

### Check how the returned data looks like

In [3]:
data

{'batchcomplete': '',
 'continue': {'sroffset': 10, 'continue': '-||'},
 'query': {'searchinfo': {'totalhits': 9878},
  'search': [{'ns': 0,
    'title': 'Mannheim',
    'pageid': 99627,
    'size': 78141,
    'wordcount': 7276,
    'snippet': '<span class="searchmatch">Mannheim</span> (German pronunciation: [ˈmanhaɪm] (listen); Palatine German: Mannem or Monnem), officially the University City of <span class="searchmatch">Mannheim</span> (German: Universitätsstadt',
    'timestamp': '2023-06-27T00:53:27Z'},
   {'ns': 0,
    'title': 'Bruno Mannheim',
    'pageid': 6788624,
    'size': 10829,
    'wordcount': 1346,
    'snippet': '&quot;Ugly&quot; <span class="searchmatch">Mannheim</span> is a supervillain appearing American comic books published by DC Comics. He is an Intergang crime boss who is the son of Moxie <span class="searchmatch">Mannheim</span> and',
    'timestamp': '2023-06-28T13:05:32Z'},
   {'ns': 0,
    'title': 'University of Mannheim',
    'pageid': 3910645,
    'size'

### Get the first 10 results

In [4]:
sr = data['query']['search']
sr

[{'ns': 0,
  'title': 'Mannheim',
  'pageid': 99627,
  'size': 78141,
  'wordcount': 7276,
  'snippet': '<span class="searchmatch">Mannheim</span> (German pronunciation: [ˈmanhaɪm] (listen); Palatine German: Mannem or Monnem), officially the University City of <span class="searchmatch">Mannheim</span> (German: Universitätsstadt',
  'timestamp': '2023-06-27T00:53:27Z'},
 {'ns': 0,
  'title': 'Bruno Mannheim',
  'pageid': 6788624,
  'size': 10829,
  'wordcount': 1346,
  'snippet': '&quot;Ugly&quot; <span class="searchmatch">Mannheim</span> is a supervillain appearing American comic books published by DC Comics. He is an Intergang crime boss who is the son of Moxie <span class="searchmatch">Mannheim</span> and',
  'timestamp': '2023-06-28T13:05:32Z'},
 {'ns': 0,
  'title': 'University of Mannheim',
  'pageid': 3910645,
  'size': 50404,
  'wordcount': 4236,
  'snippet': 'The University of <span class="searchmatch">Mannheim</span> (German: Universität <span class="searchmatch">Mannheim</spa

### Put the results in a pandas dataframe

In [5]:
df_sr = pd.DataFrame(sr)
df_sr

,ns,title,pageid,size,wordcount,snippet,timestamp
0,0,Mannheim,99627,78141,7276,"<span class=""searchmatch"">Mannheim</span> (Ger...",2023-06-27T00:53:27Z
1,0,Bruno Mannheim,6788624,10829,1346,"&quot;Ugly&quot; <span class=""searchmatch"">Man...",2023-06-28T13:05:32Z
2,0,University of Mannheim,3910645,50404,4236,"The University of <span class=""searchmatch"">Ma...",2023-03-23T09:46:11Z
3,0,Mannheim (disambiguation),2800145,1852,269,"<span class=""searchmatch"">Mannheim</span> is a...",2022-07-23T00:18:13Z
4,0,Mannheim school,310723,10292,1229,"<span class=""searchmatch"">Mannheim</span> scho...",2023-01-18T14:40:16Z
5,0,Mannheim process,3502906,3142,323,"The <span class=""searchmatch"">Mannheim</span> ...",2023-01-06T21:03:00Z
6,0,Amédée Mannheim,1843576,1634,77,"Victor Mayer Amédée <span class=""searchmatch"">...",2023-03-04T02:55:31Z
7,0,Karl Mannheim,174959,25714,3128,"Karl <span class=""searchmatch"">Mannheim</span>...",2023-06-15T03:51:44Z
8,0,Timeline of Mannheim,52266499,14445,990,"history of the city of <span class=""searchmatc...",2022-06-01T00:03:40Z
9,0,Mannheim Steamroller,582787,24923,1556,"<span class=""searchmatch"">Mannheim</span> Stea...",2023-06-20T14:28:17Z


## A simple wrapper function to search Wikipedia

In [6]:
def searchWikipedia(search_term, lang='en', n_max=100):
    base_url = f'https://{lang}.wikipedia.org/w/api.php?action=query&list=search&srsearch={search_term}&format=json'
    print(f'Visiting URL {base_url}')
    r = requests.get(base_url)
    sr = json.loads(r.text)
    df_sr = pd.DataFrame(sr['query']['search'])
    
    # get the number of search results
    n_sr = sr['query']['searchinfo']['totalhits']
    print(f'Total number of results: {n_sr}')
    
    n_max = min(n_max, n_sr)
    
    for i in range(1, int(n_max / 10) + 1):
        url = base_url + '&sroffset=' + str(i * 10)
        print(f'Visiting URL {url}')
        r = requests.get(url)
        sr = json.loads(r.text)
        df_sr = pd.concat([df_sr, pd.DataFrame(sr['query']['search'])])
        
    return df_sr.reset_index()[:n_max]

In [7]:
searchWikipedia('mannheim', lang='zh', n_max=260)

Visiting URL https://zh.wikipedia.org/w/api.php?action=query&list=search&srsearch=mannheim&format=json
Total number of results: 253
Visiting URL https://zh.wikipedia.org/w/api.php?action=query&list=search&srsearch=mannheim&format=json&sroffset=10
Visiting URL https://zh.wikipedia.org/w/api.php?action=query&list=search&srsearch=mannheim&format=json&sroffset=20
Visiting URL https://zh.wikipedia.org/w/api.php?action=query&list=search&srsearch=mannheim&format=json&sroffset=30
Visiting URL https://zh.wikipedia.org/w/api.php?action=query&list=search&srsearch=mannheim&format=json&sroffset=40
Visiting URL https://zh.wikipedia.org/w/api.php?action=query&list=search&srsearch=mannheim&format=json&sroffset=50
Visiting URL https://zh.wikipedia.org/w/api.php?action=query&list=search&srsearch=mannheim&format=json&sroffset=60
Visiting URL https://zh.wikipedia.org/w/api.php?action=query&list=search&srsearch=mannheim&format=json&sroffset=70
Visiting URL https://zh.wikipedia.org/w/api.php?action=query&li

,index,ns,title,pageid,size,wordcount,snippet,timestamp
0,0,0,曼海姆,472034,32936,1129,"曼海姆（<span class=""searchmatch"">Mannheim</span>，...",2022-10-28T17:14:27Z
1,1,0,曼海姆大学,226309,7222,531,"曼海姆大学（Universität <span class=""searchmatch"">Ma...",2022-11-08T10:49:00Z
2,2,0,耶穌會教堂 (曼海姆),4445103,2171,151,"Jesuitenkirche in <span class=""searchmatch"">Ma...",2022-07-03T10:01:09Z
3,3,0,曼海姆水塔,5062902,1791,154,"Rohrbach nach <span class=""searchmatch"">Mannhe...",2022-12-15T12:20:09Z
4,4,0,曼海姆火车总站,4032572,46186,4238,"曼海姆火车总站（德語：<span class=""searchmatch"">Mannheim<...",2022-11-27T18:19:26Z
...,...,...,...,...,...,...,...,...
248,8,0,小行星列表/243501-243600,3363463,11697,16,2010年3月12日 卡特林那 卡特林那巡天系统 小行星243535 2010 ET87 2...,2014-08-07T14:00:39Z
249,9,0,小行星列表/243001-244000,3364936,2617,35,2010年3月12日 卡特林那 卡特林那巡天系统 小行星243535 2010 ET87 2...,2013-04-16T01:24:43Z
250,0,0,轰炸乌尔姆,6558787,2124,259,War II） 美因茲（英语：Bombing of Mainz in World War I...,2022-03-28T19:22:20Z
251,1,0,大蚊属,8400984,117774,12353,"Alexander, 1933 Tipula (Lunatipula) latistyla ...",2023-06-02T06:42:30Z


## Retrieve the content of the English Wikipedia page "Mannheim"

In [8]:
url = "https://en.wikipedia.org/w/api.php?action=parse&page=Mannheim&format=json"
r = requests.get(url)
data = json.loads(r.text)

### Check how the returned data looks like

In [9]:
data['parse']

{'title': 'Mannheim',
 'pageid': 99627,
 'revid': 1162104194,
 'text': {'*': '<div class="mw-parser-output"><div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">Second-largest city in Baden-Württemberg, Germany</div>\n<style data-mw-deduplicate="TemplateStyles:r1033289096">.mw-parser-output .hatnote{font-style:italic}.mw-parser-output div.hatnote{padding-left:1.6em;margin-bottom:0.5em}.mw-parser-output .hatnote i{font-style:normal}.mw-parser-output .hatnote+link+.hatnote{margin-top:-0.5em}</style><div role="note" class="hatnote navigation-not-searchable">This article is about the city in Germany. For other uses, see <a href="/wiki/Mannheim_(disambiguation)" class="mw-disambig" title="Mannheim (disambiguation)">Mannheim (disambiguation)</a>.</div>\n<p class="mw-empty-elt">\n</p>\n<div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">City in Baden-Württemberg, Germany</div><style data-mw-deduplicate="TemplateStyles:r106

### Get the links to the Wikipedia "Mannheim" pages of other languages

In [10]:
langlinks = data['parse']['langlinks']
langlinks

[{'lang': 'af',
  'url': 'https://af.wikipedia.org/wiki/Mannheim',
  'langname': 'Afrikaans',
  'autonym': 'Afrikaans',
  '*': 'Mannheim'},
 {'lang': 'als',
  'url': 'https://als.wikipedia.org/wiki/Mannheim',
  'langname': 'Swiss German',
  'autonym': 'Alemannisch',
  '*': 'Mannheim'},
 {'lang': 'am',
  'url': 'https://am.wikipedia.org/wiki/%E1%88%9B%E1%8A%95%E1%88%83%E1%8B%AD%E1%88%9D',
  'langname': 'Amharic',
  'autonym': 'አማርኛ',
  '*': 'ማንሃይም'},
 {'lang': 'ar',
  'url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%A7%D9%86%D9%87%D8%A7%D9%8A%D9%85',
  'langname': 'Arabic',
  'autonym': 'العربية',
  '*': 'مانهايم'},
 {'lang': 'an',
  'url': 'https://an.wikipedia.org/wiki/Mannheim',
  'langname': 'Aragonese',
  'autonym': 'aragonés',
  '*': 'Mannheim'},
 {'lang': 'ast',
  'url': 'https://ast.wikipedia.org/wiki/Mannheim',
  'langname': 'Asturian',
  'autonym': 'asturianu',
  '*': 'Mannheim'},
 {'lang': 'az',
  'url': 'https://az.wikipedia.org/wiki/Manhaym',
  'langname': 'Azerbaijani',
  '

### Put the results in a pandas dataframe

In [11]:
df_langlinks = pd.DataFrame(langlinks)
df_langlinks

,lang,url,langname,autonym,*
0,af,https://af.wikipedia.org/wiki/Mannheim,Afrikaans,Afrikaans,Mannheim
1,als,https://als.wikipedia.org/wiki/Mannheim,Swiss German,Alemannisch,Mannheim
2,am,https://am.wikipedia.org/wiki/%E1%88%9B%E1%8A%...,Amharic,አማርኛ,ማንሃይም
3,ar,https://ar.wikipedia.org/wiki/%D9%85%D8%A7%D9%...,Arabic,العربية,مانهايم
4,an,https://an.wikipedia.org/wiki/Mannheim,Aragonese,aragonés,Mannheim
...,...,...,...,...,...
100,war,https://war.wikipedia.org/wiki/Mannheim,Waray,Winaray,Mannheim
101,wuu,https://wuu.wikipedia.org/wiki/%E6%9B%BC%E6%B5...,Wu Chinese,吴语,曼海姆
102,zh-yue,https://zh-yue.wikipedia.org/wiki/%E6%9B%BC%E6...,Cantonese,粵語,曼海姆
103,diq,https://diq.wikipedia.org/wiki/Mannheim,Zazaki,Zazaki,Mannheim


### Get the page content (in HTML)

In [12]:
html = data['parse']['text']['*']
html

'<div class="mw-parser-output"><div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">Second-largest city in Baden-Württemberg, Germany</div>\n<style data-mw-deduplicate="TemplateStyles:r1033289096">.mw-parser-output .hatnote{font-style:italic}.mw-parser-output div.hatnote{padding-left:1.6em;margin-bottom:0.5em}.mw-parser-output .hatnote i{font-style:normal}.mw-parser-output .hatnote+link+.hatnote{margin-top:-0.5em}</style><div role="note" class="hatnote navigation-not-searchable">This article is about the city in Germany. For other uses, see <a href="/wiki/Mannheim_(disambiguation)" class="mw-disambig" title="Mannheim (disambiguation)">Mannheim (disambiguation)</a>.</div>\n<p class="mw-empty-elt">\n</p>\n<div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">City in Baden-Württemberg, Germany</div><style data-mw-deduplicate="TemplateStyles:r1066479718">.mw-parser-output .infobox-subbox{padding:0;border:none;margin:-3px;

### Render the HTML

In [13]:
HTML(html)